In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

# Load data from CSV
file_path = 'final_manglish_transliterated.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Ensure 'commentText' column contains strings
data['transliterated_text'] = data['transliterated_text'].astype(str)

# Function to transliterate Malayalam text to English
def transliterate_malayalam_to_english(text):
    return transliterate(text, sanscript.MALAYALAM, sanscript.ITRANS)

data['transliterated_text'] = data['transliterated_text'].apply(transliterate_malayalam_to_english)

# Function to convert sentiment labels to numerical values
sentiment_dict = {
    'Positive': 0,
    'Negative': 1,
    'Not_relevant': 2,
    'Mixed Feelings': 3,
    'Neutral': 4
}

data['Sentiment_Class'] = data['Sentiment_Class'].map(sentiment_dict)

# Tokenization and sequence padding
max_words = 1000
max_seq_length = 100
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(data['transliterated_text'])

sequences = tokenizer.texts_to_sequences(data['transliterated_text'])
padded_sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post', truncating='post')

# Prepare target variable
labels = np.asarray(data['Sentiment_Class'])

# Upsample minority classes to match the size of the majority class
data_upsampled = pd.concat([
    resample(data[data['Sentiment_Class'] == sentiment_dict['Positive']], replace=True, n_samples=len(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']])),
    resample(data[data['Sentiment_Class'] == sentiment_dict['Negative']], replace=True, n_samples=len(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']])),
    resample(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']], replace=True, n_samples=len(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']])),
    resample(data[data['Sentiment_Class'] == sentiment_dict['Mixed Feelings']], replace=True, n_samples=len(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']])),
    resample(data[data['Sentiment_Class'] == sentiment_dict['Neutral']], replace=True, n_samples=len(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']]))
])

# Tokenization and sequence padding for balanced data
sequences_upsampled = tokenizer.texts_to_sequences(data_upsampled['transliterated_text'])
padded_sequences_upsampled = pad_sequences(sequences_upsampled, maxlen=max_seq_length, padding='post', truncating='post')

# Prepare target variable for upsampled data
labels_upsampled = np.asarray(data_upsampled['Sentiment_Class'])

# Split the upsampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences_upsampled, labels_upsampled, test_size=0.2, random_state=10)

# Build Bi-LSTM model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=16))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(5, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the model with early stopping
callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

history = model.fit(X_train, y_train, epochs=15, batch_size=64, validation_split=0.2, callbacks=callbacks)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Generate predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert numerical labels back to original sentiment labels
reverse_sentiment_dict = {v: k for k, v in sentiment_dict.items()}
y_test_labels = [reverse_sentiment_dict[label] for label in y_test]
y_pred_labels = [reverse_sentiment_dict[label] for label in y_pred_classes]

# Print complete classification report
print(classification_report(y_test_labels, y_pred_labels, target_names=sentiment_dict.keys()))


Epoch 1/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.3105 - loss: 1.5038 - val_accuracy: 0.5543 - val_loss: 1.1119
Epoch 2/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.5975 - loss: 1.0369 - val_accuracy: 0.6799 - val_loss: 0.8710
Epoch 3/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.7112 - loss: 0.8019 - val_accuracy: 0.7191 - val_loss: 0.7674
Epoch 4/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.7599 - loss: 0.6915 - val_accuracy: 0.7250 - val_loss: 0.7834
Epoch 5/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.7770 - loss: 0.6378 - val_accuracy: 0.7475 - val_loss: 0.7117
Epoch 6/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.7934 - loss: 0.5921 - val_accuracy: 0.7505 - val_loss: 0.7098
Epoch 7/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.8130 - loss: 0.5549 - val_accuracy: 0.7684 - val_loss: 0.6839
Epoch 8/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.8093 - loss: 0.5440 - val_acc

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

# Load data from CSV
file_path = 'final_manglish_transliterated.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Ensure 'commentText' column contains strings
data['transliterated_text'] = data['transliterated_text'].astype(str)

# Function to convert sentiment labels to numerical values
sentiment_dict = {
    'Positive': 0,
    'Negative': 1,
    'Not_relevant': 2,
    'Mixed Feelings': 3,
    'Neutral': 4
}

data['Sentiment_Class'] = data['Sentiment_Class'].map(sentiment_dict)

# Tokenization and sequence padding
max_words = 40000
max_seq_length = 200
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(data['transliterated_text'])

sequences = tokenizer.texts_to_sequences(data['transliterated_text'])
padded_sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post', truncating='post')

# Prepare target variable
labels = np.asarray(data['Sentiment_Class'])

# Upsample minority classes to match the size of the majority class
data_upsampled = pd.concat([
    resample(data[data['Sentiment_Class'] == sentiment_dict['Positive']], replace=True, n_samples=len(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']])),
    resample(data[data['Sentiment_Class'] == sentiment_dict['Negative']], replace=True, n_samples=len(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']])),
    resample(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']], replace=True, n_samples=len(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']])),
    resample(data[data['Sentiment_Class'] == sentiment_dict['Mixed Feelings']], replace=True, n_samples=len(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']])),
    resample(data[data['Sentiment_Class'] == sentiment_dict['Neutral']], replace=True, n_samples=len(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']]))
])

# Tokenization and sequence padding for balanced data
sequences_upsampled = tokenizer.texts_to_sequences(data_upsampled['transliterated_text'])
padded_sequences_upsampled = pad_sequences(sequences_upsampled, maxlen=max_seq_length, padding='post', truncating='post')

# Prepare target variable for upsampled data
labels_upsampled = np.asarray(data_upsampled['Sentiment_Class'])

# Split the upsampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences_upsampled, labels_upsampled, test_size=0.2, random_state=0)

e
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the model with early stopping
callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=callbacks)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Generate predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert numerical labels back to original sentiment labels
reverse_sentiment_dict = {v: k for k, v in sentiment_dict.items()}
y_test_labels = [reverse_sentiment_dict[label] for label in y_test]
y_pred_labels = [reverse_sentiment_dict[label] for label in y_pred_classes]

# Print complete classification report
print(classification_report(y_test_labels, y_pred_labels, target_names=sentiment_dict.keys()))


Epoch 1/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 24s 68ms/step - accuracy: 0.3237 - loss: 1.4742 - val_accuracy: 0.6256 - val_loss: 0.9481
Epoch 2/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 19s 59ms/step - accuracy: 0.7096 - loss: 0.7749 - val_accuracy: 0.8024 - val_loss: 0.6024
Epoch 3/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 19s 59ms/step - accuracy: 0.8817 - loss: 0.3933 - val_accuracy: 0.8654 - val_loss: 0.4488
Epoch 4/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 19s 59ms/step - accuracy: 0.9542 - loss: 0.1852 - val_accuracy: 0.8843 - val_loss: 0.4111
Epoch 5/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 20s 64ms/step - accuracy: 0.9786 - loss: 0.0950 - val_accuracy: 0.8905 - val_loss: 0.4142
Epoch 6/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 21s 66ms/step - accuracy: 0.9868 - loss: 0.0640 - val_accuracy: 0.8976 - val_loss: 0.4112
Epoch 7/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 20s 62ms/step - accuracy: 0.9911 - loss: 0.0418 - val_accuracy: 0.8960 - val_loss: 0.4372
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8954 - loss: 0.4434
Test Accuracy: